# NCAA Tournament

## Imports

In [26]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

from selenium import webdriver
from selenium.webdriver import Chrome
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests

import copy
import random

import multiprocessing
from joblib import Parallel, delayed

## Functions

In [27]:
#Webdriver parser used to extract the brackets
def parse2(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get(url)
    sleep(3)
    sourceCode = driver.page_source
    return sourceCode

In [28]:
def score_bracket(bracket, results):
    points = 0
    for key in bracket.keys():
        if key != "points":
            bracket[key] = set(bracket[key])
            results[key] = set(results[key])
            points += rounds_points[key] * len(bracket[key] & results[key])
    return points

In [29]:
def comp_brackets(results, brackets, round_index=0, index=0):
	bracket_scores = [[0 for x in range(len(brackets))] for y in range(len(brackets))]
	if results["1"][0] != '':
		ordered = []
		results_copy = results.copy()
		for b in range(len(brackets)):
			bracket = brackets[b]
			score = score_bracket(bracket, results_copy)
			ordered.append((b, score))
		ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
		for x in range(len(ordered)):
			bracket_scores[ordered[x][0]][x] = 1
		return bracket_scores
	else:
		if index == int(rounds[round_index]) - 1:
			next_round_index = round_index + 1
			next_index = 0
		else:
			next_round_index = round_index
			next_index = index + 1


		if results[rounds[round_index]][index] == '':
			options = [results[rounds[round_index - 1]][index * 2], results[rounds[round_index - 1]][index * 2 + 1]]
			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[0]
			branch1 = comp_brackets(results_copy, brackets, next_round_index, next_index)

			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[1]
			branch2 = comp_brackets(results_copy, brackets, next_round_index, next_index)
			total_branch = np.add(branch1, branch2)
			return total_branch
		else:
			return comp_brackets(results, brackets, next_round_index, next_index)

def comp_brackets_weighted(results, brackets, team_to_seed, round_index=0, index=0, liklihood = 1):
	bracket_scores = [[0 for x in range(len(brackets))] for y in range(len(brackets))]
	if results["1"][0] != '':
		ordered = []
		results_copy = results.copy()
		for b in range(len(brackets)):
			bracket = brackets[b]
			score = score_bracket(bracket, results_copy)
			ordered.append((b, score))
		ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
		for x in range(len(ordered)):
			bracket_scores[ordered[x][0]][x] = 1 * liklihood
		return bracket_scores
	else:
		if index == int(rounds[round_index]) - 1:
			next_round_index = round_index + 1
			next_index = 0
		else:
			next_round_index = round_index
			next_index = index + 1


		if results[rounds[round_index]][index] == '':
			options = [results[rounds[round_index - 1]][index * 2], results[rounds[round_index - 1]][index * 2 + 1]]
			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[0]
			update = seed_vs_seed[team_to_seed[options[0]]][team_to_seed[options[1]]]
			branch1 = comp_brackets_weighted(results_copy, brackets, team_to_seed, next_round_index, next_index, liklihood * update)

			results_copy = copy.deepcopy(results)
			results_copy[rounds[round_index]][index] = options[1]
			update = seed_vs_seed[team_to_seed[options[1]]][team_to_seed[options[0]]]
			branch2 = comp_brackets_weighted(results_copy, brackets, team_to_seed, next_round_index, next_index, liklihood * update)
			total_branch = np.add(branch1, branch2)
			return total_branch
		else:
			return comp_brackets_weighted(results, brackets, team_to_seed, next_round_index, next_index, liklihood)

In [30]:
def comp_MC(results, brackets, time_allocated = 10, samples = 1000, std_thresh = 0.1):
    start_time = time.time()
    trials = []
    while time.time() - start_time < time_allocated:
        bracket_scores = [np.array([0 for x in range(len(brackets))]) for y in range(len(brackets))]
        for trial in range(samples):
            results_copy = copy.deepcopy(results)
            for round_index in range(len(results_copy)-1):
                round = rounds[round_index]
                for index in range(len(results_copy[round])):
                    if results_copy[round][index] == '':
                        options = [results_copy[rounds[round_index - 1]][index * 2], results_copy[rounds[round_index - 1]][index * 2 + 1]]
                        option = options[random.choice([0,1])]
                        results_copy[round][index] = option
            ordered = []
            for b in range(len(brackets)):
                bracket = brackets[b]
                score = score_bracket(bracket, results_copy)
                ordered.append((b, score))
            ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
            for x in range(len(ordered)):
                bracket_scores[ordered[x][0]][x] += 1
        trials.append(np.array(bracket_scores) / samples)

        if len(trials) > 10:
            std = np.array(trials)
            test = std[0,:,0]
            max_win = np.argmax(test)
            std = std[:, max_win, 0]
            std = np.std(std) * 100
            print(std)
            if std < std_thresh:
                break
    trials = np.array(trials)
    std = trials.std(axis=0)
    trials = trials.mean(axis=0)
    return trials, std

def comp_MC_weighted(results, brackets, team_to_seed, time_allocated = 10, samples = 1000, std_thresh = 0.1):
    start_time = time.time()
    trials = []
    while time.time() - start_time < time_allocated:
        bracket_scores = [np.array([0 for x in range(len(brackets))]) for y in range(len(brackets))]
        for trial in range(samples):
            results_copy = copy.deepcopy(results)
            for round_index in range(len(results_copy)-1):
                round = rounds[round_index]
                for index in range(len(results_copy[round])):
                    if results_copy[round][index] == '':
                        options = [results_copy[rounds[round_index - 1]][index * 2], results_copy[rounds[round_index - 1]][index * 2 + 1]]
                        weights = [seed_vs_seed[team_to_seed[options[0]]][team_to_seed[options[1]]], seed_vs_seed[team_to_seed[options[1]]][team_to_seed[options[0]]]]
                        option = options[random.choices([0,1], weights=weights)[0]]
                        results_copy[round][index] = option
            ordered = []
            for b in range(len(brackets)):
                bracket = brackets[b]
                score = score_bracket(bracket, results_copy)
                ordered.append((b, score))
            ordered = sorted(ordered, key= lambda x: x[1], reverse=True)
            for x in range(len(ordered)):
                bracket_scores[ordered[x][0]][x] += 1
        trials.append(np.array(bracket_scores) / samples)
        if len(trials) > 10:
            std = np.array(trials)
            test = std[0,:,0]
            max_win = np.argmax(test)
            std = std[:, max_win, 0]
            std = np.std(std) * 100
            print(std)
            if std < std_thresh:
                break
    trials = np.array(trials)
    std = trials.std(axis=0)
    trials = trials.mean(axis=0)
    return trials, std

In [31]:
#Gets the actual games and results that have been played
def get_so_far(url):
    print(url)
    bracket = {"64": [], "32": [], "16": [], "8": [], "4": [], "2": [], "1": []}
    data = requests.get(url)
    soup = BeautifulSoup(data.text, features='html.parser')
    index = 0
    team_to_seed = {}
    for x in range(1, 64):
        class_i = "matchup m_" + str(x)
        mydivs = soup.find_all("div", {"class": class_i})
        if mydivs == []:
            class_ii = class_i + " userPickable"
            mydivs = soup.find_all("div", {"class": class_ii})
            if mydivs == []:
                class_ii = class_i + " homeOnBottom" + " userPickable"
                mydivs = soup.find_all("div", {"class": class_ii})
            mydivs = mydivs[0].find_all("span", {"class": "name"})
            team1 = mydivs[0].text.strip()
            team2 = mydivs[1].text.strip()
            team3 = mydivs[2].text.strip()
            team4 = mydivs[3].text.strip()
        else:
            mydivs = mydivs[0].find_all("div", {"class": "slots"})
            team1 = mydivs[0].find_all("div", {"class": "slot s_1"})
            team1 = team1[0].find_all("span", {"class": "name"})
            team1 = team1[0].text.strip()

            team2 = mydivs[0].find_all("div", {"class": "slot s_2"})
            team2 = team2[0].find_all("span", {"class": "name"})
            team2 = team2[0].text.strip()
        if x <= 32:
            bracket["64"].append(team1)
            bracket["64"].append(team2)
            team_to_seed[team1] = seed_order[index]
            team_to_seed[team2] = seed_order[index + 1]
            index += 2
            if index == 16:
                index = 0
        elif x <= 48:
            bracket["32"].append(team1)
            bracket["32"].append(team3)
        elif x <= 56:
            bracket["16"].append(team1)
            bracket["16"].append(team3)
        elif x <= 60:
            bracket["8"].append(team1)
            bracket["8"].append(team3)
        elif x <= 62:
            bracket["4"].append(team1)
            bracket["4"].append(team3)
        elif x <= 63:
            bracket["2"].append(team1)
            bracket["2"].append(team3)

    mydivs = soup.find_all("div", {"class": "slot userPickable"})
    if len(mydivs) == 0:
        mydivs = soup.find_all("span", {"class": "actual winner"})
    teams = mydivs[0].find_all("span", {"class": "name"})
    team1 = teams[0].text.strip()
    # team2 = teams[1].text.strip()
    bracket["1"].append(team1)
    bracket["points"] = 1920
    return bracket, team_to_seed

#Gets the picks for a bracket for each game
def get_bracket(url):
    bracket = {"64": [], "32": [], "16": [], "8": [], "4": [], "2": [], "1": [], "points": 0}
    data = requests.get(url)
    soup = BeautifulSoup(data.text, features='html.parser')
    classes = []
    for x in range(1, 64):
        class_i = "matchup m_" + str(x)

        mydivs = soup.find_all("div", {"class": class_i})

        if mydivs == []:
            class_ii = class_i + " userPickable"
            mydivs = soup.find_all("div", {"class": class_ii})

            if mydivs == []:
                class_ii = class_i + " homeOnBottom" + " userPickable"
                mydivs = soup.find_all("div", {"class": class_ii})
            try:
                mydivs = mydivs[0].find_all("span", {"class": "name"})
            except:
                return {'64': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '32': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',
                               '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '16': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1'],
                        '8': ['1', '1', '1', '1', '1', '1', '1', '1'], '4': ['1', '1', '1', '1'], '2': ['1', '1'],
                        '1': ['1'], 'points': 260}

            team1 = mydivs[0].text.strip()
            team2 = mydivs[1].text.strip()
            team3 = mydivs[2].text.strip()
            team4 = mydivs[3].text.strip()
        else:
            mydivs = mydivs[0].find_all("div", {"class": "slots"})
            team1 = mydivs[0].find_all("div", {"class": "slot s_1"})
            team1 = team1[0].find_all("span", {"class": "name"})
            team1 = team1[0].text.strip()

            team2 = mydivs[0].find_all("div", {"class": "slot s_2"})
            team2 = team2[0].find_all("span", {"class": "name"})
            team2 = team2[0].text.strip()

        if x <= 32:
            bracket["64"].append(team1)
            bracket["64"].append(team2)
        elif x <= 48:
            bracket["32"].append(team2)
            bracket["32"].append(team4)
            if team1 == team2:
                bracket["points"] += 10
            if team3 == team4:
                bracket["points"] += 10
        elif x <= 56:
            bracket["16"].append(team2)
            bracket["16"].append(team4)
            if team1 == team2:
                bracket["points"] += 20
            if team3 == team4:
                bracket["points"] += 20
        elif x <= 60:
            bracket["8"].append(team2)
            bracket["8"].append(team4)
            if team1 == team2:
                bracket["points"] += 40
            if team3 == team4:
                bracket["points"] += 40
        elif x <= 62:
            bracket["4"].append(team2)
            bracket["4"].append(team4)
            if team1 == team2:
                bracket["points"] += 80
            if team3 == team4:
                bracket["points"] += 80
        elif x <= 63:
            bracket["2"].append(team2)
            bracket["2"].append(team4)
            if team1 == team2:
                bracket["points"] += 160
            if team3 == team4:
                bracket["points"] += 160
    try:
        mydivs = soup.find_all("div", {"class": "slot userPickable"})
        teams = mydivs[0].find_all("span", {"class": "name"})
    except:
        try:
            mydivs = soup.find_all("div", {"class": "slot wWinner userPickable"})
            teams = mydivs[0].find_all("span", {"class": "name"})
        except:
            mydivs = soup.find_all("div", {"class": "slot s_2 correct wWinner userPickable"})
            teams = mydivs[0].find_all("span", {"class": "name"})
    team1 = teams[0].text.strip()
    team2 = teams[1].text.strip()
    bracket["1"].append(team2)
    if team1 == team2:
        bracket["points"] += 320

    return bracket

#Gets the IDs of all the participants in a group
def get_group(group):
    IDs = []
    soup = BeautifulSoup(parse2(group))
    mydivs = soup.find_all("table", {"class": "type_entries"})
    entries = mydivs[0].find_all("a", {"class": "entry"})
    for entry in entries:
        IDs.append((entry.get('href'), entry.text.strip()))
    return IDs

## Constants

In [32]:
base_bracket_url = "https://fantasy.espn.com/tournament-challenge-bracket/2023/en/"

rounds = ["64", "32", "16", "8", "4", "2", "1"]

rounds_points = {"64": 0, "32": 10, "16": 20, "8": 40, "4": 80, "2": 160, "1": 320}

seed_vs_seed = {'1': {'1': 0.5, '2': 0.539, '3': 0.634, '4': 0.711, '5': 0.839, '6': 0.706, '7': 0.857, '8': 0.798, '9': 0.901, '10': 0.857, '11': 0.556, '12': 1.0, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.993}, '2': {'1': 0.461, '2': 0.5, '3': 0.603, '4': 0.444, '5': 0.167, '6': 0.722, '7': 0.694, '8': 0.444, '9': 0.5, '10': 0.645, '11': 0.833, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.938, '16': 0.5}, '3': {'1': 0.366, '2': 0.397, '3': 0.5, '4': 0.625, '5': 0.5, '6': 0.576, '7': 0.611, '8': 0.5, '9': 0.5, '10': 0.692, '11': 0.679, '12': 0.5, '13': 0.5, '14': 0.847, '15': 0.5, '16': 0.5}, '4': {'1': 0.289, '2': 0.556, '3': 0.375, '4': 0.5, '5': 0.563, '6': 0.333, '7': 0.333, '8': 0.364, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.69, '13': 0.79, '14': 0.5, '15': 0.5, '16': 0.5}, '5': {'1': 0.161, '2': 0.833, '3': 0.5, '4': 0.438, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.671, '13': 0.842, '14': 0.5, '15': 0.5, '16': 0.5}, '6': {'1': 0.294, '2': 0.278, '3': 0.424, '4': 0.667, '5': 0.5, '6': 0.5, '7': 0.625, '8': 0.5, '9': 0.5, '10': 0.6, '11': 0.634, '12': 0.5, '13': 0.5, '14': 0.875, '15': 0.5, '16': 0.5}, '7': {'1': 0.143, '2': 0.306, '3': 0.389, '4': 0.667, '5': 0.5, '6': 0.333, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.601, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '8': {'1': 0.202, '2': 0.556, '3': 0.5, '4': 0.636, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.518, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '9': {'1': 0.099, '2': 0.5, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.482, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '10': {'1': 0.143, '2': 0.355, '3': 0.308, '4': 0.5, '5': 0.5, '6': 0.4, '7': 0.399, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '11': {'1': 0.444, '2': 0.167, '3': 0.321, '4': 0.5, '5': 0.5, '6': 0.366, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.9, '15': 0.5, '16': 0.5}, '12': {'1': 0.1, '2': 0.5, '3': 0.5, '4': 0.31, '5': 0.329, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.75, '14': 0.5, '15': 0.5, '16': 0.5}, '13': {'1': 0.5, '2': 0.5, '3': 0.5, '4': 0.21, '5': 0.158, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.25, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '14': {'1': 0.5, '2': 0.5, '3': 0.153, '4': 0.5, '5': 0.5, '6': 0.125, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.1, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '15': {'1': 0.5, '2': 0.063, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}, '16': {'1': 0.007, '2': 0.5, '3': 0.5, '4': 0.5, '5': 0.5, '6': 0.5, '7': 0.5, '8': 0.5, '9': 0.5, '10': 0.5, '11': 0.5, '12': 0.5, '13': 0.5, '14': 0.5, '15': 0.5, '16': 0.5}}

seed_order = ['1', '16', "8", "9", "5", "12", "4", "13", "6", "11", "3", "14", "7", "10", "2", "15"]

## Main

In [33]:
# This is the only thing you need to change. Just change to the url of your group
group_url = "https://fantasy.espncdn.com/tournament-challenge-bracket/2023/en/group?groupID=1030576"

IDs = get_group(group_url)

results, team_to_seed = get_so_far(base_bracket_url + IDs[0][0])

brackets = []
for x in tqdm(range(len(IDs))):
    bracket = get_bracket(base_bracket_url + IDs[x][0])
    brackets.append(bracket)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 111.0.5563
[WDM] - Get LATEST chromedriver version for 111.0.5563 google-chrome
[WDM] - Driver [C:\Users\aidan\.wdm\drivers\chromedriver\win32\111.0.5563.64\chromedriver.exe] found in cache
C:\Users\aidan\AppData\Local\Temp\ipykernel_8984\4167765881.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


https://fantasy.espn.com/tournament-challenge-bracket/2023/en/entry?entryID=89255659


100%|██████████| 46/46 [00:50<00:00,  1.09s/it]


In [34]:
# Runs weighted Monte Carlo Simulation.
# Weights are proportional to how likely that bracket is to happen based on historical matchups.

# time_allocated = how long you want to run the code for before getting results as Monte Carlo is an approximation
# std_thresh = standard deviation of the win probabilities that you want to hit. If it gets under this threshold before the 
# time threshold is hit. The program will break. 

total, std = comp_MC_weighted(results, brackets, team_to_seed, samples = 100000, time_allocated=120, std_thresh=0.1)
total = total / np.sum(total[0]) * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values('1st', ascending=False)
df

,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,...,37th,38th,39th,40th,41th,42th,43th,44th,45th,46th
Tri_Sal-23-E4W1-E,12.383000,0.694333,2.173333,0.066000,2.005667,0.983000,5.119667,0.132333,2.842333,1.290333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-E2M1-M,11.796667,9.810333,0.967667,3.723667,0.224000,5.608667,0.786000,1.979333,0.460667,3.558333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-E2W2-E,8.101333,3.306000,3.152667,0.414000,2.968000,6.892667,3.177000,5.686333,2.123667,2.273667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-E6M2-E,7.018333,0.638000,4.228667,0.579333,4.701333,2.630667,4.380000,1.827333,4.058667,1.744000,...,12.119000,0.000000,8.278000,0.000000,4.054000,0.000000,0.000000,0.000000,6.416333,0.000000
Tri_Sal-23-S1W4-S,5.635333,5.534333,4.595333,3.928000,6.960000,0.410000,1.453333,1.128667,3.089333,1.682667,...,3.001667,0.761000,1.248000,0.000000,0.000000,0.000000,13.611667,0.000000,0.000000,0.000000
Tony_Sal-23-S1W2-S,5.039000,7.444333,3.695333,4.762667,0.126333,4.529667,3.458333,3.014333,3.265000,3.399333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-E2M1-E,5.000667,0.435000,5.255333,0.226667,3.615667,0.820667,10.235000,2.752333,5.287667,3.962333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-E2M2-M,4.521667,4.753667,4.315000,6.234333,3.789333,3.871000,0.507333,2.474333,4.786667,1.857667,...,3.462667,4.321000,8.874333,4.243333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tony_Sal-23-S1M1-M,4.343667,11.796667,7.171000,0.067333,1.996333,0.060667,7.475667,2.918000,2.135333,3.566000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tony_Sal-23-S1M1-S,4.277000,4.100333,3.862000,6.541333,2.609000,1.935000,4.488000,10.243000,2.728667,2.324667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [35]:
# Unweighted Monted Carlo Bracket. Means all possible brackets are equally likely (1 beating a 16 is the same odds as 16 beating a 1)
total, std = comp_MC(results, brackets, samples = 100000, time_allocated=120, std_thresh=0.1)
total = total / np.sum(total[0]) * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values('1st', ascending=False)
df

,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th,...,37th,38th,39th,40th,41th,42th,43th,44th,45th,46th
Tri_Sal-23-E4W1-E,24.017333,1.939500,5.005000,0.465000,4.349000,1.999333,4.481667,0.242167,5.712667,0.750667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-E6M2-E,14.081667,1.288500,9.952667,0.786833,10.413333,3.302333,5.168333,0.743333,6.400000,0.648500,...,10.304500,0.000000,6.959167,0.000000,0.397333,0.000000,0.000000,0.000000,2.727500,0.000000
Tri_Sal-23-E2W2-E,10.325000,1.163833,4.880500,0.649167,3.985833,11.248833,3.054167,4.249333,3.551333,5.249000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-S1W4-S,7.278167,4.043833,5.714167,1.542167,3.042667,0.638833,2.227833,0.279167,3.481167,4.147667,...,1.760500,1.034000,0.767167,0.000000,0.000000,0.000000,21.527167,0.000000,0.000000,0.000000
Tony_Sal-23-E6W2-W,6.103000,4.319833,2.373667,3.122667,6.360667,4.011500,0.797833,1.477667,3.097000,1.501667,...,2.058000,1.356333,0.689000,15.039500,7.542000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-S1W3-S,4.168333,2.812167,3.937333,1.253667,2.823333,1.595333,0.327667,0.175833,1.165833,0.178667,...,5.235833,2.046333,2.356833,1.377333,1.019000,0.000000,0.000000,0.000000,28.731333,0.000000
Tri_Sal-23-E2M2-M,4.032000,3.828500,4.342167,8.346167,3.626167,4.355500,0.900500,2.369500,8.182000,1.830167,...,0.274833,1.253333,7.164000,6.161833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-E2W2-W,3.673667,10.717000,0.215333,6.714667,1.516667,4.313500,8.187167,1.366167,3.547167,3.242000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-E2M1-M,3.650333,5.134667,0.142833,2.845667,0.200000,5.132833,0.475167,2.029333,0.094667,1.870167,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tri_Sal-23-S1W4-W,3.157667,6.343833,0.707167,5.479167,0.903500,2.188167,1.379500,2.760167,0.355833,3.599333,...,1.528667,1.760500,1.034000,0.767167,0.000000,0.000000,0.000000,21.527167,0.000000,0.000000


In [ ]:
# Runs weighted Simulation.
# Weights are proportional to how likely that bracket is to happen based on historical matchups.
# Checks all possible bracket permutations remaining.
# Takes too long before ~Elite 8 or most of the was through sweet 16 as it is O(2^n)
total, std = comp_brackets_weighted(results, brackets, team_to_seed)
total = total / np.sum(total[0]) * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values('1st', ascending=False)
df

In [ ]:
# Runs unweighted Simulation.
# Weights are proportional to how likely that bracket is to happen based on historical matchups.
# Checks all possible bracket permutations remaining.
# Takes too long before ~Elite 8 or most of the was through sweet 16 as it is O(2^n)
total, std = comp_brackets(results, brackets)
total = total / np.sum(total[0]) * 100
columns = ["1st", "2nd", "3rd"]
for x in range(3, len(IDs)):
	columns.append(str(x+1)+"th")
rows = [entry[1] for entry in IDs]

df = pd.DataFrame(total, columns = columns, index = rows)
df = df.sort_values('1st', ascending=False)
df